In [1]:
num_epochs = 10
batch_size = 32
best_accuracy = 0

In [2]:
import torch
import torch.nn as nn
import numpy as np
import os
import pandas as pd
import dask.dataframe as dd
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
from IPython.display import clear_output
import time

In [3]:
class MyCNN(nn.Module):
    def __init__(self, num_layers=8, input_features=34, output_features=34):
        super(MyCNN, self).__init__()
        
        self.conv_layers = nn.ModuleList()
        self.conv_layers.append(nn.Conv1d(in_channels=4, out_channels=256, kernel_size=3, padding=1))
        
        for _ in range(1, num_layers - 1):
            self.conv_layers.append(nn.Conv1d(in_channels=256, out_channels=256, kernel_size=3, padding=1))
        
        self.conv_layers.append(nn.Conv1d(in_channels=256, out_channels=1, kernel_size=1, padding=0))
        
        self.activation = nn.LeakyReLU(negative_slope=0.01)
    def forward(self, x):
        for i, conv in enumerate(self.conv_layers):
            if(i < len(self.conv_layers) - 1):
                x = self.activation(conv(x))
            else:
                x = conv(x)
        x = x.squeeze(1)
        print("x.shape:",x.shape)
        
        return x

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [5]:
model = torch.load('E:/專題/model/discard_model_84.375.pth')
#model.load_state_dict(torch.load('E:/專題/model/discard_model_weights.pth'))
model.to(device)
loss_criterion  = nn.CrossEntropyLoss()
model.eval()

MyCNN(
  (conv_layers): ModuleList(
    (0): Conv1d(4, 256, kernel_size=(3,), stride=(1,), padding=(1,))
    (1-7): 7 x Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
    (8): Conv1d(256, 1, kernel_size=(1,), stride=(1,))
  )
  (activation): LeakyReLU(negative_slope=0.01)
)

In [6]:
class MyDataset(Dataset):
    def __init__(self, file_path, chunk_size):
        self.file_path = file_path
        self.chunk_size = chunk_size
        
        self.columns = pd.read_csv(self.file_path, nrows=0).columns.str.strip().str.replace('\xa0', ' ').tolist()
        self.train_columns = [col for col in self.columns if col != 'Discard']  # 排除 'Discard' 欄位
        self.num_samples = sum(1 for _ in open(self.file_path, encoding='utf-8')) - 1  # 計算總樣本數
        
    def __getitem__(self, idx):
        chunk_start = idx // self.chunk_size * self.chunk_size
        df = pd.read_csv(self.file_path, skiprows=chunk_start + 1, nrows=self.chunk_size, header=None, encoding='utf-8')  # 跳過標題和之前的行

        df.columns = self.columns
        
        # 確保 Reward 欄位存在
        if 'Discard' not in df.columns:
            raise KeyError(f"Chunk starting at row {chunk_start + 1} does not contain 'Discard' column.")
        
            
        sample_idx = idx % self.chunk_size
        
        train_data = df[self.train_columns].iloc[sample_idx].values  # 提取除了 'Reward' 之外的欄位
        
        value_data = df['Discard'].iloc[sample_idx]  # 提取 'Reward' 欄位
        
        return torch.tensor(train_data, dtype=torch.float32), torch.tensor(value_data, dtype=torch.float32)
        
    def __len__(self):
        return self.num_samples

In [7]:
"""
讀取csv
"""
file_path = 'E:/專題/data/2022/DiscardDataAll.csv'
dataset = MyDataset(file_path, chunk_size=50000)  # 每次只加載 50000 行
test_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [8]:
correct = 0
total = 0
with torch.no_grad():
    for train_data, value_data in test_dataloader:
        torch.autograd.set_detect_anomaly(True)
        channel4_tensor = torch.zeros((batch_size, 4, 34))
        for j in range(1, 5):
            channel4_tensor[:, j-1, :] = (train_data >= j).int()
        channel4_tensor = channel4_tensor.to(device)
        value_data = value_data.to(device)
        
        output = model(channel4_tensor)
        output = output.float()
        value_data = value_data.long()
        probabilities = F.softmax(output, dim=1)
        predicted_labels = torch.argmax(probabilities, dim=1)
        correct_predictions = (predicted_labels == value_data).sum().item()
        accuracy = (correct_predictions / value_data.size(0))*100
        
        correct += correct_predictions
        total += value_data.size(0)
        totalAccuracy = (correct/total)*100
        
        loss = loss_criterion(output, value_data)
        
        current_loss = loss
        
        clear_output()
        
        print(predicted_labels)
        print(value_data)
        print(f"Correct Predictions: {correct_predictions}")
        print(f"Total Predictions: {value_data.size(0)}")
        print(f"Cur Accuracy: {accuracy:.4f}","%")
        print(f"Total Accuracy: {totalAccuracy:.4f}","%")
        print(f"loss: {current_loss}")
        print(f"best accuracy: {best_accuracy}")
        
        if accuracy >= best_accuracy:
            best_accuracy = accuracy

tensor([25, 27,  6, 11, 25, 12, 22,  0, 10, 12, 13,  8, 29, 29, 33,  9, 29, 29,
        25, 18, 29, 29, 31, 29, 29, 25, 30, 14, 29, 25, 30, 10],
       device='cuda:0')
tensor([ 1, 27,  6, 11, 19, 19, 24,  0, 30,  1, 13,  8, 27, 28, 33,  9, 11, 27,
         7, 33, 31, 32, 31, 29, 29,  9, 10,  9, 29, 18,  3, 10],
       device='cuda:0')
Correct Predictions: 13
Total Predictions: 32
Cur Accuracy: 40.6250 %
Total Accuracy: 45.0856 %
loss: 1.8404752016067505
best accuracy: 87.5


KeyboardInterrupt: 